## The Pipeline
The pipeline in PowerShell is the mechanism by which objects generated by commands can be provided to other commands as input. Commands themselves do not understand *how* to pass objects between each other, only how to accept the appropriate object and how to generate an object as output. The PowerShell engine behind the scenes handles the passing of objects from the first command, along the pipeline, to the last command.

Commands can only accept objects of certain types as input:

In [1]:
Get-Command -ParameterType String | Select-Object -First 10


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        help                                                          
Function        mkdir                                                         
Function        oss                                                           
Function        TabExpansion2                                                 
Cmdlet          Add-History                                        7.4.1.500  Microsoft.PowerShell…
Cmdlet          Add-Member                                         7.0.0.0    Microsoft.PowerShell…
Cmdlet          Add-Type                                           7.0.0.0    Microsoft.PowerShell…
Cmdlet          Clear-History                                      7.4.1.500  Microsoft.PowerShell…
Cmdlet          Clear-Variable                                     7.0.0.0    Microsoft.PowerShell…
Cmdlet       

And output objects of certain types:

In [65]:
Get-Command | Where-Object {$_.OutputType -match "String" } | Select-Object -First 10


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Cmdlet          Convert-Path                                       7.0.0.0    Microsoft.PowerShell…
Cmdlet          ConvertTo-Csv                                      7.0.0.0    Microsoft.PowerShell…
Cmdlet          ConvertTo-Html                                     7.0.0.0    Microsoft.PowerShell…
Cmdlet          ConvertTo-Json                                     7.0.0.0    Microsoft.PowerShell…
Cmdlet          ConvertTo-Xml                                      7.0.0.0    Microsoft.PowerShell…
Cmdlet          Get-Clipboard                                      7.0.0.0    Microsoft.PowerShell…
Cmdlet          Get-Command                                        7.4.1.500  Microsoft.PowerShell…
Cmdlet          Get-Content                                        7.0.0.0    Microsoft.PowerShell…
Cmdlet          Get-Date     

The pipeline uses the pipe operator "|" to join commands together, passing the output of one command as the input to another command.

In [3]:
Get-Process code | Select-Object -ExpandProperty Modules | Select-Object FileName | Select-Object -Last 10


FileName
--------
C:\Windows\System32\win32u.dll
C:\Windows\System32\GDI32.dll
C:\Windows\System32\gdi32full.dll
C:\Windows\System32\IMM32.DLL
C:\Windows\system32\uxtheme.dll
C:\Windows\system32\mswsock.dll
C:\Windows\System32\SHELL32.dll
C:\Windows\System32\shcore.dll
C:\Windows\System32\ADVAPI32.dll
C:\Windows\SYSTEM32\kernel.appcore.dll



Error: Command failed: SubmitCode: Get-Process code | Select-Object -ExpandProperty M ...

## Parameter Binding
When objects are sent down the pipeline, PowerShell attempts to associate the object with command parameters that accept pipeline input; this is known as _parameter binding_.

In [81]:
Get-Help Get-Process -Parameter * | Where-Object { $_.pipelineInput -Like 'true*' } | Select-Object Name, PipelineInput, Type


name        pipelineInput         type
----        -------------         ----
Id          true (ByPropertyName) @{name=int[]}
InputObject true (ByValue)        @{name=Process[]}
Name        true (ByPropertyName) @{name=string[]}



The _Get-Process_ cmdlet shown above has several parameters that accept pipeline input. The **Name** parameter for example, accepts pipelineinput as long the object is of type **\<string\>**.

_ByValue_ means the parameter accepts pipeline input if the object matches the expect type or can be converted to that type.

_ByPropertyName_ means the parameter will accept input if the object has a property that matches the parameter name.

In the below code, the custom object 'Foo' is created, with a single property 'bar', that has the value 'powershell'. As the command Get-Process has no parameter that accepts a custom object as the input type, or an object with the property called 'bar', the pipeline execution fails:

In [100]:
$Foo = [pscustomobject]@{ Bar="powershell" };
"The type of Foo is: " + ($Foo).GetType().FullName;
$Foo | Get-Process

Custom Objects type is: System.Management.Automation.PSCustomObject
Get-Process: 
Line |
   3 |  $CustomObject | Get-Process
     |                  ~~~~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.


Error: Command failed: SubmitCode: $CustomObject = [pscustomobject]@{ Foo="powershell ...

Notice how the below works thoough; because _Get-Process_ does have a parameter called Name that accepts input ByPropertyName, if we change the property of the custom object to be called **Name**, _Get-Process_ returns the expected result.

In [2]:
$Foo = [pscustomobject]@{ Name="powershell" };
"The type of Foo is: " + ($Foo).GetType().FullName;
$Foo | Get-Process

The type of Foo is: System.Management.Automation.PSCustomObject

 NPM(K)    PM(M)      WS(M)     CPU(s)      Id  SI ProcessName
 ------    -----      -----     ------      --  -- -----------
     26    53.20      57.09       0.53   14696  29 powershell



## Debugging the Pipeline
The _Trace-Command_ cmdlet creates a subpipeline and runs the provided expression inside this subpipeline, meaning each step of the expression ca be inspected. In the below output, it can be seen that the Foo PSCustomObject is repeatedly matched against the _Get-Process_ cmdlets parameters, looking to see if the value type or property name matches. 

In [8]:
$Foo = [pscustomobject]@{ Bar="powershell" };
Trace-Command ParameterBinding -Expression {$Foo | Get-Process} -PSHost

DEBUG: 2024-03-11 19:33:31.5858 ParameterBinding Information: 0 : BIND NAMED cmd line args [Get-Process]
DEBUG: 2024-03-11 19:33:31.5876 ParameterBinding Information: 0 : BIND POSITIONAL cmd line args [Get-Process]
DEBUG: 2024-03-11 19:33:31.5877 ParameterBinding Information: 0 : MANDATORY PARAMETER CHECK on cmdlet [Get-Process]
DEBUG: 2024-03-11 19:33:31.5878 ParameterBinding Information: 0 : CALLING BeginProcessing
DEBUG: 2024-03-11 19:33:31.5882 ParameterBinding Information: 0 : BIND PIPELINE object to parameters: [Get-Process]
DEBUG: 2024-03-11 19:33:31.5883 ParameterBinding Information: 0 :     PIPELINE object TYPE = [System.Management.Automation.PSCustomObject]
DEBUG: 2024-03-11 19:33:31.5884 ParameterBinding Information: 0 :     RESTORING pipeline parameter's original values
DEBUG: 2024-03-11 19:33:31.5885 ParameterBinding Information: 0 :     Parameter [InputObject] PIPELINE INPUT ValueFromPipeline NO COERCION
DEBUG: 2024-03-11 19:33:31.5886 ParameterBinding Information: 0 :   

Error: Command failed: SubmitCode: $Foo = [pscustomobject]@{ Bar="powershell" }; ...